In [1]:
# to start with deep learning for genomics
# deep learning to learn a model directly from data.

In [2]:
##setup tensorflow v1
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [3]:
!wget -c https://repo.anaconda.com/archive/Anaconda3-2019.10-Linux-x86_64.sh
!chmod +x Anaconda3-2019.10-Linux-x86_64.sh
!bash ./Anaconda3-2019.10-Linux-x86_64.sh -b -f -p /usr/local
!conda install -y -c deepchem -c rdkit -c conda-forge -c omnia deepchem-gpu=2.3.0
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

--2020-07-25 20:17:10--  https://repo.anaconda.com/archive/Anaconda3-2019.10-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 530308481 (506M) [application/x-sh]
Saving to: ‘Anaconda3-2019.10-Linux-x86_64.sh’

Anaconda3-2019.10-L 100%[===================>] 505.74M   155MB/s    in 3.2s    

2020-07-25 20:17:14 (156 MB/s) - ‘Anaconda3-2019.10-Linux-x86_64.sh’ saved [530308481/530308481]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: \ | / - \ | / - \ | / - \ | / - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _ipyw_jlab_nb_ext_conf==0.1.0=py37_0
    - _libgcc_mutex==0.1=main
    - alabaster==0.7.12=py37_0
    - anaconda-client==1.7.2=py37_0
    - anaconda-navigator==1.9.7=py37_0
    - ana

In [4]:
import deepchem as dc
dc.__version__ #should match with the installed

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



'2.3.0'

In [97]:
import numpy as np

In [5]:
# We will try to train a model that predicts those labels based on the sequence of each
# segment.

In [75]:
#upload data to drive as zip file and then unzip

In [73]:
#extractall() with any argument will extract everything in the same directory 
# this is what is expected
import zipfile
with zipfile.ZipFile('tocolabch6.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [39]:
import tensorflow as tf
import deepchem.models.tensorgraph.layers as layers

In [107]:
#del model

In [108]:
# dna-transcription factor binding with chromation accesibility
model = dc.models.TensorGraph(batch_size=1000)
features = layers.Feature(shape=(None, 101, 4))
#accessibility = layers.Feature(shape=(None, 1)) #data is added later
labels = layers.Label(shape=(None, 1))
weights = layers.Weights(shape=(None, 1))

In [109]:
prev = features
for i in range(3):
  prev = layers.Conv1D(filters=15, kernel_size=10,activation=tf.nn.relu, padding='same',in_layers=prev)
  prev = layers.Dropout(dropout_prob=0.5, in_layers=prev)

In [110]:
logits = layers.Dense(out_channels=1, in_layers=layers.Flatten(prev))
output = layers.Sigmoid(logits)
model.add_output(output)

In [111]:
loss = layers.SigmoidCrossEntropy(in_layers=[labels, logits])
weighted_loss = layers.WeightedError(in_layers=[loss, weights])
model.set_loss(weighted_loss)

In [84]:
train = dc.data.DiskDataset('train_dataset')
valid = dc.data.DiskDataset('valid_dataset')
metric = dc.metrics.Metric(dc.metrics.roc_auc_score)

Loading dataset from disk.
Loading dataset from disk.


In [85]:
# data loaded after an hour of debugging , noob! Training below till 200 epochs

In [100]:
for i in range(20):
  model.fit(train, nb_epoch=10)
  print (i)
  print('training: ', model.evaluate(train, [metric]))
  print('validation: ',model.evaluate(valid, [metric]))

In [88]:
#model is overfitting; 
# there are fundamental limits to how well this model can ever work.
# accessibility, methylation, shape, the presence of other molecules, etc.
# Any model that ignores those factors will be limited in how accurate its predictions

In [90]:
# incorporating chromatin accesibility
# HEPG2 cells in experiment human liver cancer cell line. 
# Hep G2 is an immortal cell line which was derived from the
# liver tissue of a 15-year-old African American adolescent 
# boy with a well-differentiated hepatocellular carcinoma.

In [91]:
span_accessibility = {}
for line in open('accessibility.txt'):
  fields = line.split()
  span_accessibility[fields[0]] = float(fields[1])

In [112]:
accessibility = layers.Feature(shape=(None, 1)) #added earlier

In [113]:
logits = layers.Dense(out_channels=1, in_layers=layers.Flatten(prev))
prev = layers.Concat([layers.Flatten(prev), accessibility])
logits = layers.Dense(out_channels=1, in_layers=prev)

In [114]:
def generate_batches(dataset, epochs):
  for epoch in range(epochs):
    for X, y, w, ids in dataset.iterbatches(batch_size=1000,pad_batches=True):
      yield {features: X,accessibility: np.array([span_accessibility[id] for id in ids]),labels: y,weights: w}

In [115]:
for i in range(5):
  print (i)
  model.fit_generator(generate_batches(train, 10))
  print('training:       ',model.evaluate_generator(generate_batches(train, 1), [metric],labels=[labels], weights=[weights]))
  print('validation:       ',model.evaluate_generator(generate_batches(valid, 1), [metric],labels=[labels], weights=[weights]))
  print ('\n')

0
computed_metrics: [0.6577745346466051]
training:        {'roc_auc_score': 0.6577745346466051}
computed_metrics: [0.5651583404819402]
validation:        {'roc_auc_score': 0.5651583404819402}


1
computed_metrics: [0.7478063028874063]
training:        {'roc_auc_score': 0.7478063028874063}
computed_metrics: [0.7049789167559497]
validation:        {'roc_auc_score': 0.7049789167559497}


2
computed_metrics: [0.7921473429370036]
training:        {'roc_auc_score': 0.7921473429370036}
computed_metrics: [0.7221432433412369]
validation:        {'roc_auc_score': 0.7221432433412369}


3
computed_metrics: [0.8171611369178701]
training:        {'roc_auc_score': 0.8171611369178701}
computed_metrics: [0.7260813853462388]
validation:        {'roc_auc_score': 0.7260813853462388}


4
computed_metrics: [0.8233329379270488]
training:        {'roc_auc_score': 0.8233329379270488}
computed_metrics: [0.7273662420267963]
validation:        {'roc_auc_score': 0.7273662420267963}




In [117]:
#ok its training, for more epochs

In [118]:
for i in range(5,20):
  print (i)
  model.fit_generator(generate_batches(train, 10))
  print('training:       ',model.evaluate_generator(generate_batches(train, 1), [metric],labels=[labels], weights=[weights]))
  print('validation:       ',model.evaluate_generator(generate_batches(valid, 1), [metric],labels=[labels], weights=[weights]))
  print ('\n')

5
computed_metrics: [0.8396511513009136]
training:        {'roc_auc_score': 0.8396511513009136}
computed_metrics: [0.7267753613516226]
validation:        {'roc_auc_score': 0.7267753613516226}


6
computed_metrics: [0.8473051048773355]
training:        {'roc_auc_score': 0.8473051048773355}
computed_metrics: [0.7179987114577859]
validation:        {'roc_auc_score': 0.7179987114577859}


7
computed_metrics: [0.8486629687401014]
training:        {'roc_auc_score': 0.8486629687401014}
computed_metrics: [0.7306290349121628]
validation:        {'roc_auc_score': 0.7306290349121628}


8
computed_metrics: [0.85626574195966]
training:        {'roc_auc_score': 0.85626574195966}
computed_metrics: [0.7180632580993086]
validation:        {'roc_auc_score': 0.7180632580993086}


9
computed_metrics: [0.8578158543335699]
training:        {'roc_auc_score': 0.8578158543335699}
computed_metrics: [0.7309098327245894]
validation:        {'roc_auc_score': 0.7309098327245894}


10
computed_metrics: [0.8543191094

In [120]:
#clearly overfitting, need more data/features, may be dna structural information
# validation max at 0.74 and then decreases.

In [122]:
#del model

In [119]:
#RNA Interference: another case of using ML in genomics

In [121]:
#It also is a powerful tool for biology and medicine. It lets you temporarily “turn off ”
#any gene you want.

In [123]:
# Model to predict the power so different siRNA at 
# how effective it is at silencing its target gene.
# Small values indicate
# ineffective molecules, while larger values indicate more effective ones. The model
# takes the sequence as input and tries to predict the effectiveness.
# siRNA Regression

model = dc.models.TensorGraph()
features = layers.Feature(shape=(None, 21, 4))
labels = layers.Label(shape=(None, 1))
prev = features
for i in range(2):
  prev = layers.Conv1D(filters=10, kernel_size=10,activation=tf.nn.relu, padding='same',in_layers=prev)
  prev = layers.Dropout(dropout_prob=0.3, in_layers=prev)

output = layers.Dense(out_channels=1, activation_fn=tf.sigmoid,in_layers=layers.Flatten(prev))

model.add_output(output)

loss = layers.ReduceMean(layers.L2Loss(in_layers=[labels, output]))

model.set_loss(loss)

In [124]:
train = dc.data.DiskDataset('train_siRNA')
valid = dc.data.DiskDataset('valid_siRNA')
metric = dc.metrics.Metric(dc.metrics.pearsonr, mode='regression')

Loading dataset from disk.
Loading dataset from disk.


In [125]:
for i in range(20):
  print(i)
  model.fit(train, nb_epoch=10)
  print('training:   ',model.evaluate(train, [metric]))
  print('validation: ',model.evaluate(valid, [metric]))
  print('\n')

0
computed_metrics: [(0.6115135956022426, 1.7443417077325025e-224)]
training:    {'pearsonr': (0.6115135956022426, 1.7443417077325025e-224)}
computed_metrics: [(0.5399365161826473, 7.274577139356909e-20)]
validation:  {'pearsonr': (0.5399365161826473, 7.274577139356909e-20)}


1
computed_metrics: [(0.6741400532538732, 7.621697196890229e-290)]
training:    {'pearsonr': (0.6741400532538732, 7.621697196890229e-290)}
computed_metrics: [(0.616864286873979, 5.645644959325156e-27)]
validation:  {'pearsonr': (0.616864286873979, 5.645644959325156e-27)}


2
computed_metrics: [(0.6978189368163467, 5.4513e-319)]
training:    {'pearsonr': (0.6978189368163467, 5.4513e-319)}
computed_metrics: [(0.6267066640303587, 4.969433424826179e-28)]
validation:  {'pearsonr': (0.6267066640303587, 4.969433424826179e-28)}


3
computed_metrics: [(0.7120352003850609, 0.0)]
training:    {'pearsonr': (0.7120352003850609, 0.0)}
computed_metrics: [(0.6276759222214924, 3.8933936909877905e-28)]
validation:  {'pearsonr': (0

In [126]:
# validation correlation maxed out at 0.634, train corre increases ; classic case of overfitting

In [127]:
# with the amount of data available, this result is okay in my opinion

In [128]:
# completed chapter 6